In [1]:
import numpy as np
from src.utils import *
from tqdm import tqdm

unary_operators=[np.sin, np.cos, np.exp, np.abs, np.log, np.tan]
binary_operators=[np.add, np.subtract, np.multiply, np.divide]


operators = unary_operators + binary_operators

In [ ]:
problem_0 = np.load("data/problem_4.npz")
x = problem_0["x"]
y = problem_0["y"]
print("x.shape:", x.shape)
print("y.shape:", y.shape)

In [ ]:
NUM_POPULATION = 100
NUM_GENERATIONS = 500
MAX_DEPTH_INITIAL = 5
DEDUPE_INTERVAL = 15
ELITISM = True
ELITE_COUNT = 3
TOURNAMENT_SUBSET_SIZE = 3
TOURNAMENT_WINNER_SIZE = 20
XOVER = 0.8
MUTATION = 0.8
BREED_NEW = 15
KILL_AGE = 16
CONSTANT_FIT_INTERVAL = 30
CONSTANT_FIT_ITERATION = 6

my_population = create_population(NUM_POPULATION,MAX_DEPTH_INITIAL,x.shape[0])
assign_population_fitness(my_population,x,y)
print(f" Population size: {len(my_population)}")
print(f"Population mean fitness: {calculate_mean_fitness(my_population)}")
print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")

# Sort the population by fitness (ascending), then take the top 5
top_5_individuals = top_n_individuals(my_population, 5)

# Print the top 5 individuals with the minimum fitness
for i, individual in enumerate(top_5_individuals, 1):
    print(f"Top {i}: Fitness = {individual.fitness}: Genome = {individual.genome}")

In [ ]:
assign_population_fitness(my_population,x,y)
best_result = top_n_individuals(my_population, 1)[0]
for generation in tqdm(range(NUM_GENERATIONS)):
    num_xover=0
    num_mut=0
    num_succesfull_mut=0
    age_population(my_population)
    

    # Kill eldest individuals
    kill_eldest(my_population, KILL_AGE)
    # print("Population size before killing constants: ", len(my_population))
    
    # print("Population size after killing constants: ", len(my_population))


    best_ind = tournament_selection(my_population,TOURNAMENT_SUBSET_SIZE,TOURNAMENT_WINNER_SIZE,ELITISM=ELITISM, elite_count=ELITE_COUNT)

    
    for _ in range(BREED_NEW): # CHECK
        if random.random() < XOVER:
            # Xover the best individual and replace worst individuals
            parent1 = my_population[best_ind[np.random.choice(len(best_ind))]]
            parent2 = my_population[best_ind[np.random.choice(len(best_ind))]]
            child1, child2 = crossover(parent1,parent2)
            my_population.append(child1)
            my_population.append(child2)
            num_xover+=1

        if random.random() < MUTATION:
            # Mutate best individuals and replace worst individuals
            parent = my_population[best_ind[np.random.choice(len(best_ind))]]
            
            child, success = mutation_w_sa(parent, x.shape[0], x, y)
            num_mut+=1
            if success:                
                my_population.append(child)
                num_succesfull_mut+=1

    assign_population_fitness(my_population,x,y)
    # print(f"xover: {num_xover}, mut: {num_mut}, succesfull mut: {num_succesfull_mut}")
    simplify_constant_population(my_population)
    # Update elites age as 0
    elites = top_n_individuals(my_population, ELITE_COUNT)
    if best_result.fitness > elites[0].fitness:
        best_result = elites[0]
        print(f"New best result found: {best_result}")
    
    for elite in elites:
        my_population.remove(elite)
        elite.age = 0
        my_population.append(elite)

        

    # Deduplicate every few iterations
    if generation % DEDUPE_INTERVAL == DEDUPE_INTERVAL-1:
        init_population = len(my_population)
        my_population = deduplicate_population(my_population)
        dedup_population = len(my_population)
        simplify_operation_population(my_population)
        simplified_operations = len(my_population)
        kill_constant(my_population)
        no_constant = len(my_population)

        print(f"Initial: {init_population}, Deduplicated: {init_population-dedup_population}, Operation simplified: {dedup_population-simplified_operations}, Constant: {simplified_operations-no_constant}, Final: {no_constant}")
        print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")
        print(f"Population mean fitness: {calculate_mean_fitness(my_population)}")
        print()
        best_5 = top_n_individuals(my_population, 5)
        for ind in range(len(best_5)):
            print(f"{ind}. Fitness: {best_5[ind].fitness:.3f}, Genome: {best_5[ind].genome}, Age: {best_5[ind].age}")
    #     # Fit constants of every individual
    if generation % CONSTANT_FIT_INTERVAL == CONSTANT_FIT_INTERVAL-1:
        print(f"Generation {generation}: Mean fitness {calculate_mean_fitness(my_population)}")
        for ind in range(len(my_population)):
            stronger_inv = fit_constants(my_population[ind],CONSTANT_FIT_ITERATION, x, y)
            my_population[ind] = stronger_inv
        print(f"Population mean fitness after constant fit: {calculate_mean_fitness(my_population)}")
        best_5 = top_n_individuals(my_population, 5)
        for ind in range(len(best_5)):
            print(f"{ind}. Fitness: {best_5[ind].fitness:.3f}, Genome: {best_5[ind].genome}, Age: {best_5[ind].age}")
      

my_population = deduplicate_population(my_population)

In [ ]:
assign_population_fitness(my_population,x,y)
# Sort the population by fitness (ascending), then take the top 5
top_5_individuals = top_n_individuals(my_population, 15)

# Print the best individual
print(f"Best individual: Fitness = {best_result.fitness}: Genome = {best_result.genome}")
# Print the top 5 individuals with the minimum fitness in population
for i, individual in enumerate(top_5_individuals, 1):
    print(f"Top {i}: Fitness = {individual.fitness}: Genome = {individual.genome}")

# Print the mean fitness of the population
print("Mean fitness of the population: ", calculate_mean_fitness(my_population))
print("Population size: ", len(my_population))
print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")

In [5]:
no_constant_optimization = my_population

In [ ]:
NUM_POPULATION = 100
NUM_GENERATIONS = 500
MAX_DEPTH_INITIAL = 5
DEDUPE_INTERVAL = 15
ELITISM = True
ELITE_COUNT = 3
TOURNAMENT_SUBSET_SIZE = 3
TOURNAMENT_WINNER_SIZE = 15
XOVER = 0.8
MUTATION = 0.8
BREED_NEW = 15
KILL_AGE = 15
CONSTANT_FIT_INTERVAL = 30
CONSTANT_FIT_ITERATION = 5

my_population = create_population(NUM_POPULATION,MAX_DEPTH_INITIAL,x.shape[0])
assign_population_fitness(my_population,x,y)
print(f" Population size: {len(my_population)}")
print(f"Population mean fitness: {calculate_mean_fitness(my_population)}")
print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")

# Sort the population by fitness (ascending), then take the top 5
top_5_individuals = top_n_individuals(my_population, 5)

# Print the top 5 individuals with the minimum fitness
for i, individual in enumerate(top_5_individuals, 1):
    print(f"Top {i}: Fitness = {individual.fitness}: Genome = {individual.genome}")

In [ ]:
assign_population_fitness(my_population,x,y)
best_result = top_n_individuals(my_population, 1)[0]
for generation in tqdm(range(NUM_GENERATIONS)):
    num_xover=0
    num_mut=0
    num_succesfull_mut=0
    age_population(my_population)
    

    # Kill eldest individuals
    kill_eldest(my_population, KILL_AGE)
    # print("Population size before killing constants: ", len(my_population))
    
    # print("Population size after killing constants: ", len(my_population))


    best_ind = tournament_selection(my_population,TOURNAMENT_SUBSET_SIZE,TOURNAMENT_WINNER_SIZE,elitism=ELITISM, elite_count=ELITE_COUNT)

    
    for _ in range(BREED_NEW): # CHECK
        if random.random() < XOVER:
            # Xover the best individual and replace worst individuals
            parent1 = my_population[best_ind[np.random.choice(len(best_ind))]]
            parent2 = my_population[best_ind[np.random.choice(len(best_ind))]]
            child1, child2 = crossover(parent1,parent2)
            my_population.append(child1)
            my_population.append(child2)
            num_xover+=1

        if random.random() < MUTATION:
            # Mutate best individuals and replace worst individuals
            parent = my_population[best_ind[np.random.choice(len(best_ind))]]
            
            child, success = mutation_w_sa(parent, x.shape[0], x, y)
            num_mut+=1
            if success:                
                my_population.append(child)
                num_succesfull_mut+=1

    assign_population_fitness(my_population,x,y)
    # print(f"xover: {num_xover}, mut: {num_mut}, succesfull mut: {num_succesfull_mut}")
    simplify_constant_population(my_population)
    # Update elites age as 0
    elites = top_n_individuals(my_population, ELITE_COUNT)
    if best_result.fitness > elites[0].fitness:
        best_result = elites[0]
        print(f"New best result found: {best_result}")
    
    for elite in elites:
        my_population.remove(elite)
        elite.age = 0
        my_population.append(elite)

        

    # Deduplicate every few iterations
    if generation % DEDUPE_INTERVAL == DEDUPE_INTERVAL-1:
        init_population = len(my_population)
        my_population = deduplicate_population(my_population)
        dedup_population = len(my_population)
        simplify_operation_population(my_population)
        simplified_operations = len(my_population)
        kill_constant(my_population)
        no_constant = len(my_population)

        print(f"Initial: {init_population}, Deduplicated: {init_population-dedup_population}, Operation simplified: {dedup_population-simplified_operations}, Constant: {simplified_operations-no_constant}, Final: {no_constant}")
        print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")
        print(f"Population mean fitness: {calculate_mean_fitness(my_population)}")
        print()
        best_5 = top_n_individuals(my_population, 5)
        for ind in range(len(best_5)):
            print(f"{ind}. Fitness: {best_5[ind].fitness:.3f}, Genome: {best_5[ind].genome}, Age: {best_5[ind].age}")
        # Fit constants of every individual
    if generation % CONSTANT_FIT_INTERVAL == CONSTANT_FIT_INTERVAL-1:
        print(f"Generation {generation}: Mean fitness {calculate_mean_fitness(my_population)}")
        for ind in range(len(my_population)):
            stronger_inv = fit_constants(my_population[ind],CONSTANT_FIT_ITERATION, x, y)
            my_population[ind] = stronger_inv
        print(f"Population mean fitness after constant fit: {calculate_mean_fitness(my_population)}")
        best_5 = top_n_individuals(my_population, 5)
        for ind in range(len(best_5)):
            print(f"{ind}. Fitness: {best_5[ind].fitness:.3f}, Genome: {best_5[ind].genome}, Age: {best_5[ind].age}")
        
        
my_population = deduplicate_population(my_population)

In [ ]:
assign_population_fitness(my_population,x,y)
# Sort the population by fitness (ascending), then take the top 5
top_5_individuals = top_n_individuals(my_population, 15)

# Print the best individual
print(f"Best individual: Fitness = {best_result.fitness}: Genome = {best_result.genome}")
# Print the top 5 individuals with the minimum fitness in population
for i, individual in enumerate(top_5_individuals, 1):
    print(f"Top {i}: Fitness = {individual.fitness}: Genome = {individual.genome}")

# Print the mean fitness of the population
print("Mean fitness of the population: ", calculate_mean_fitness(my_population))
print("Population size: ", len(my_population))
print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")